In [5]:
import socket 
from _thread import *
import sys
from collections import defaultdict as df
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer
import time

In [6]:
class Server:
    def __init__(self):
        self.rooms = df(list)
        self.server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        self.server.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)

In [4]:
model = pickle.load(open(r"C:\Users\selvi\Desktop\clone to vs code\CyberBullying-Detection-in-Hinglish-Languages-Using-Machine-Learning-\Safe_Chat\LinearSVC.pkl", 'rb'))

In [19]:
       def accept_connections(self, ip_address, port):
            self.ip_address = ip_address
            self.port = port
            self.server.bind((self.ip_address, int(self.port)))
            self.server.listen(100)

            while True:
                 connection, address = self.server.accept()
                 print(str(address[0]) + ":" + str(address[1]) + " Connected")

                 start_new_thread(self.clientThread, (connection,))

            self.server.close()


In [20]:
    def clientThread(self, connection):
        user_id = connection.recv(1024).decode().replace("User ", "")
        room_id = connection.recv(1024).decode().replace("Join ", "")

        if room_id not in self.rooms:
            connection.send("New Group created".encode())
        else:
            connection.send("Welcome to chat room".encode())

        self.rooms[room_id].append(connection)

        while True:
            try:
                message = connection.recv(1024)
                pred=0
                if message:
                    if str(message.decode()) == "FILE":
                        self.broadcastFile(connection, room_id, user_id)

                    else:
                        # print('Am I fucking here')
                        pred=self.prettyPrinter(str(message.decode()))
                        message_to_send = "<" + str(user_id) + "> " + message.decode()
                        self.broadcast(message_to_send, connection, room_id,pred)

                else:
                    self.remove(connection, room_id)
            except Exception as e:
                print(repr(e))
                print("Client disconnected earlier")
                break

In [21]:
    def prettyPrinter(self,data_1):
         # List of stopwords 
        my_file = open(r"C:\Users\selvi\Desktop\clone to vs code\CyberBullying-Detection-in-Hinglish-Languages-Using-Machine-Learning-\Safe_Chat\stopwords.txt", "r")
        content = my_file.read()
        content_list = content.split("\n")
        my_file.close()
        tfidf_vector =  TfidfVectorizer(stop_words = content_list, lowercase = True,vocabulary=pickle.load(open(r"C:\Users\selvi\Desktop\clone to vs code\CyberBullying-Detection-in-Hinglish-Languages-Using-Machine-Learning-\Safe_Chat\tfidf_vector_vocabulary.pkl", "rb")))
        data_2=tfidf_vector.fit_transform([data_1])
        print(data_2)
        pred = model.predict(data_2)
        print(pred)
        if pred==0:
            print('Non bullying')
            return pred
        else: 
            print("Stop bullying people and behave decently.")
            return pred

In [23]:
    def broadcast(self, message_to_send, connection, room_id,pred):
        for client in self.rooms[room_id]:
            if client != connection:
                try:
                    if pred==0:
                        client.send(message_to_send.encode())
                    else :
                        client.send("Bullying message detected it has been hidden".encode())
                except:
                    client.close()
                    self.remove(client, room_id)

In [24]:
    def remove(self, connection, room_id):
        if connection in self.rooms[room_id]:
            self.rooms[room_id].remove(connection)

In [38]:
if __name__ == "__main__":
    ip_address = "127.0.0.1"
    port = 12345
    s = Server()
    s.accept_connections(ip_address, port)

AttributeError: 'Server' object has no attribute 'accept_connections'

In [34]:
import socket
import select
import sys
import time
import pickle
import os
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer

model = pickle.load(open(r"C:\Users\selvi\Desktop\clone to vs code\CyberBullying-Detection-in-Hinglish-Languages-Using-Machine-Learning-\Safe_Chat\LinearSVC.pkl", 'rb'))
server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
IP_address = "127.0.0.1"
port = 12345

server.connect((IP_address, port))
print("Connected To server")

user_id = input("Type user id: ")
room_id = input("Type room id: ")

server.send(str.encode(user_id))
time.sleep(0.1)
server.send(str.encode(room_id))

def prettyPrinter(self,data_1):
         # List of stopwords 
    my_file = open(r"C:\Users\selvi\Desktop\clone to vs code\CyberBullying-Detection-in-Hinglish-Languages-Using-Machine-Learning-\Safe_Chat\stopwords.txt", "r")
    content = my_file.read()
    content_list = content.split("\n")
    my_file.close()
    tfidf_vector =  TfidfVectorizer(stop_words = content_list, lowercase = True,vocabulary=pickle.load(open(r"C:\Users\selvi\Desktop\clone to vs code\CyberBullying-Detection-in-Hinglish-Languages-Using-Machine-Learning-\Safe_Chat\tfidf_vector_vocabulary.pkl", "rb")))
    data_2=tfidf_vector.fit_transform([data_1])
    print(data_2)
    pred = model.predict(data_2)
    print(pred)
    if pred==0:
        print('Non bullying')
        return pred
    else: 
        # print("Stop bullying people and behave decently. If you do this again we will block you.")
        print("Bullying message detected it has been hidden")
        return pred

while True:
    socket_list = [sys.stdin, server]

    read_socket, write_socket, error_socket = select.select(socket_list, [], [])

    for socks in read_socket:
        if socks == server:
            message = socks.recv(1024)
            
            print(str(message.decode()))

            if str(message.decode()) == "FILE":
                file_name = socks.recv(1024).decode()
                lenOfFile = socks.recv(1024).decode()
                send_user = socks.recv(1024).decode()

                if os.path.exists(file_name):
                    os.remove(file_name)

                print(file_name, lenOfFile, send_user)

                total = 0
                with open(file_name, 'wb') as file:
                    while str(total) != lenOfFile:
                        data = socks.recv(1024)
                        total = total + len(data)     
                        file.write(data)
                print("<" + str(send_user) + "> " + file_name + " sent")
                       
            else:
                print(message.decode())

        else:
            message = sys.stdin.readline()

            if str(message) == "FILE\n":
                file_name = input("Enter the file name : ")
                server.send("FILE".encode())
                time.sleep(0.1)
                server.send(str("client_" + file_name).encode())
                time.sleep(0.1)
                server.send(str(os.path.getsize(file_name)).encode())
                time.sleep(0.1)

                file = open(file_name, "rb")
                data = file.read(1024)
                while data:
                    server.send(data)
                    data = file.read(1024)
                sys.stdout.write("<You>")
                sys.stdout.write("File sent successfully\n")
                sys.stdout.flush()

            else:
                server.send(message.encode())
                sys.stdout.write("<You>")
                sys.stdout.write(message)
                sys.stdout.flush()
server.close()



Connected To server


OSError: [WinError 10038] An operation was attempted on something that is not a socket